<a href="https://colab.research.google.com/github/aashu-0/Food101/blob/main/test101_examples.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [12]:
import torch
import torchvision
import matplotlib.pyplot as plt

from torch import nn
from torchvision import transforms


# installing torchmetrics and torchinfo
try:
  from torchmetrics import Accuracy
  from torchinfo import summary
except:
  print(f'[INFO] Installing torchmetrics and torchinfo')
  !pip install torchmetrics
  !pip install torchinfo


In [13]:
weights = torchvision.models.EfficientNet_B2_Weights.DEFAULT
effnetb2_transforms = weights.transforms()
model = torchvision.models.efficientnet_b2(weights=weights)

model.classifier

Sequential(
  (0): Dropout(p=0.3, inplace=True)
  (1): Linear(in_features=1408, out_features=1000, bias=True)
)

In [14]:
def create_effnetb2_model(num_classes: int,
                          seed: int=42):

  # get pretrained weights
  weights = torchvision.models.EfficientNet_B2_Weights.DEFAULT

  #effnetb2 transforms
  effnetb2_transforms = weights.transforms()

  # effnet model
  model = torchvision.models.efficientnet_b2(weights=weights)

  # freeze all the base layers
  for parm in model.parameters():
    parm.requires_grad = False

  # change classifier head
  model.classifier = nn.Sequential(
      nn.Dropout(p=0.3, inplace=True),
      nn.Linear(in_features=1408,
                out_features= num_classes)
  )

  return model, effnetb2_transforms

In [15]:
# model and transforms instance

effnetb2_model, effnetb2_transforms = create_effnetb2_model(
    num_classes=101,
    seed=42
)

In [16]:
from torchvision import datasets

# create a data folder
from pathlib import Path
data_dir = Path('data')

# download food101 test data
test_data=datasets.Food101(root=data_dir,
                           split='test',
                           transform = effnetb2_transforms,
                           download=True)
len(test_data)

25250

In [17]:
import os
import random
import shutil

# paths
base_dir = "data/food-101/images"
examples_dir = "data/food-101/examples"


os.makedirs(examples_dir, exist_ok=True)

# iterate through each food
for food_class in os.listdir(base_dir):
  class_path = os.path.join(base_dir, food_class)

  if os.path.isdir(class_path):
    images = os.listdir(class_path)

    if images:
      random_image = random.choice(images)
      src_path = os.path.join(class_path, random_image)
      dest_path = os.path.join(examples_dir, f"{food_class}.jpg")

      shutil.copy(src_path, dest_path)

print(f"Successfully copied 1 image from each category to {examples_dir}")

Successfully copied 1 image from each category to data/food-101/examples


In [18]:
import os
import shutil
from zipfile import ZipFile

#paths
examples_dir = "data/food-101/examples"
zip_path = "data/food-101/examples.zip"

# remove any exists if exists
if os.path.exists(zip_path):
  os.remove(zip_path)


shutil.make_archive(zip_path.replace(".zip", ""), 'zip', examples_dir)

# download
try:
  from google.colab import files
  files.download(zip_path)
except:
  # if not in colab
  print(f"Zip file created: {zip_path}. Please download manually.")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>